# Import DF

In [ ]:
import pandas as pd

In [ ]:
PATH_TO_OLD_CSV = '/content/submission_17_40.csv'
PATH_TO_OUTPUT_CSV = '/content/submission_modified'

df = pd.read_csv(PATH_TO_OLD_CSV)

In [ ]:
df.head()

,ImageId,EncodedPixels,ClassId,AttributesIds
0,165d068e46d8eaa97392d5db91620721,697344,-1,NaN
1,b1dfbc0da1055c82a599e6bb777bdb81,247589 19 248602 42 249613 59 250623 77 251625...,10,NaN
2,e79958a7625419a99dfc40935ad3ae6e,742400,-1,NaN
3,5a90282a3570b920d4e28f0de0a71df4,786432,-1,NaN
4,dd4525a10b6110df29f0a6b19e6b924b,362882 5 363550 26 364224 39 364900 47 365577 ...,10,NaN


# Count prediction results over test Images

In [ ]:
# 15_36 - after 2000 epochs with allowed dresses(10) & Sleeves(31)
print(df.groupby('ClassId')['ImageId'].nunique())

ClassId
-1     2450
 1        1
 6      127
 8        6
 10     555
 31      72
Name: ImageId, dtype: int64


# attribudeIds - replace None with empty string


In [ ]:
# attribudeIds is empty string instead of None
df["AttributesIds"].replace({None: '1'}, inplace=True)

ClassId
-1     2771
 1       21
 4        1
 6      397
 8       12
 33       1
Name: ImageId, dtype: int64


# Fix Run Lentgh Encoding

## RLE encoding has even number of organs

In [ ]:
# RLE encoding has even number of organs
modifyMe = df['EncodedPixels'].str.count(' ') % 2 == 0 # if we have even number of spaces, we'll have un-even number of values 
df.loc[modifyMe, 'EncodedPixels'] = df['EncodedPixels'][modifyMe].values + ' 0'

In [ ]:
# get max lenght of EncodedPixels cell

max_len = df.EncodedPixels.str.len().max()

for entry in df["EncodedPixels"]:
    strArr = entry.split(' ')
    if len(strArr) % 2 == 1:
        entry = entry + ' 1'

    print(entry)
    break



print(max_len)

12052992 1
34029.0


## Overlapping RLE

In [ ]:
 # Function to remove overlapping portion of masks (keep the portion with higher score)

def remove_overlap(masks):
    n_masks = masks.shape[2]
 
    for i in range(n_masks):
        good_mask = masks[:, :, i]
        for j in range(i + 1, n_masks):
        fix_mask = masks[:, :, j]
        overlapping = (good_mask == 1) & (fix_mask == 1)
        if np.any(overlapping):
            print('Overlapping mask detected!')
            fix_mask[overlapping] = 0
            masks[:, :, j] = fix_mask
    return masks
   

# get max sum of EncodedPixels cell


In [ ]:
# get max sum of EncodedPixels cell

max_len = df.EncodedPixels.len().max()

print(max_len)

AttributeError: ignored

# Check for duplicated values

In [ ]:
# DataFrame.duplicated(subset=None, keep='first')
df_no_copies = df.drop_duplicates(subset = 'ImageId', keep = 'first')

In [ ]:
df_no_copies.head()

,ImageId,EncodedPixels,ClassId,AttributesIds
0,165d068e46d8eaa97392d5db91620721,697344 0,-1,1
1,b1dfbc0da1055c82a599e6bb777bdb81,247589 19 248602 42 249613 59 250623 77 251625...,10,1
2,e79958a7625419a99dfc40935ad3ae6e,742400 0,-1,1
3,5a90282a3570b920d4e28f0de0a71df4,786432 0,-1,1
4,dd4525a10b6110df29f0a6b19e6b924b,362882 5 363550 26 364224 39 364900 47 365577 ...,10,1


# Generate Output CSV

In [ ]:
df.to_csv(PATH_TO_OUTPUT_CSV, index=False)

In [ ]:
df_no_copies.to_csv(PATH_TO_OUTPUT_CSV + 'noDups', index=False)